This Notebook Illustrates How to Constantly Read Data from an RSS Feed and Invoke WXO Skills with the Data Collected
The idea is to collect inputs from an RSS feed (DownDetector alerts), filter the alerts, and for any qualifying alerts, invoke a WXO skill with details of the alerts.

In [1]:
import feedparser

In [2]:
import requests
import json
import base64
import os
import time

In [3]:
Tenant_ID = "20250212-1521-3150-30c9-e789cad9cae1"
api_key = "c5jsf7CBqlU_kPW8Hj80VnU13RX2ezGPTs6Fyg0"

In [4]:
TOKEN_FILE = "bearer_token_telus.json"


def get_bearer_token(api_key):
    token_data = load_token()

    if token_data and not is_token_expired(token_data["expires_at"]):
        print("Using cached token.")
        return token_data["token"]

    print("Generating new token...")
    url = "https://iam.platform.saas.ibm.com/siusermgr/api/1.0/apikeys/token"
    payload = {"apikey": api_key}
    response = requests.post(url, json=payload)

    if response.status_code == 200:
        response_data = response.json()
        token = response_data.get("token")
        expires_in = response_data.get("expires_in")

        if token:
            save_token(token, expires_in)
            return token
        else:
            print("Token not found in response.")
    else:
        print(f"Failed to retrieve token, status code: {response.status_code}")
        return None


def save_token(token, expires_in):
    expiration_time = int(time.time()) + expires_in
    token_data = {"token": token, "expires_at": expiration_time}
    with open(TOKEN_FILE, "w") as f:
        json.dump(token_data, f)


def load_token():
    if os.path.exists(TOKEN_FILE):
        with open(TOKEN_FILE, "r") as f:
            return json.load(f)
    return None


def is_token_expired(expiration_time):
    return int(time.time()) > expiration_time


token = get_bearer_token(api_key)

Generating new token...
Failed to retrieve token, status code: 400


In [5]:
def invoke_skill_chat_response(token):
    url = f"https://api.dl.watson-orchestrate.ibm.com:443/instances/{Tenant_ID}/v1/skills/_personal_/trial-generative__latest__add_1//prompts/add_1/generation/text"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}",
        "callbackUrl": "callback",
    }

    data = {"n1": "10", "n2": "30", "output1": " "}

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_json = response.json()
        print(response_json)
        generated_text = response_json.get("generated_text", "0")
        try:
            response_num = int(generated_text)
        except ValueError:
            response_num = 0
        return response_num
    else:
        print(f"POST request failed: {response.status_code}")
        try:
            print(response.json())
        except:
            print(response.text)
        return None

In [7]:
def read_rss_feed(feed_url):
    print("====================READING RSS FEED=======================")
    # Parse the Yahoo News RSS feed
    feed = feedparser.parse(feed_url)

    print("=====================DATA COLLECTED FROM RSS FEED================")
    # List all the items present in the feed
    for item in feed.entries[:3]:
        print(item.title)
        print(item.link)
        print("------------------------")


while True:
    # Read from RSS feed
    data = read_rss_feed(feed_url="https://www.yahoo.com/news/rss")

    print("=====================INVOKE WXO SKILL================")

    # Invoke WxO Skill with input data
    token = get_bearer_token(api_key=api_key)
    print("=====================WXO output================")
    wxo_response = invoke_skill_chat_response(
        token=token
    )  # --------> This will be replaced with a skill in Wxo that sends message to google chat. 'data' collected from RSS will be supplied to this skill.

    time.sleep(30)

====================READING RSS FEED=======================
=====================DATA COLLECTED FROM RSS FEED================
=====================INVOKE WXO SKILL================
Generating new token...
Failed to retrieve token, status code: 400
=====================WXO output================
POST request failed: 401
{'message': 'wxO Unauthorized- kid not found in the token', 'code': 401}


KeyboardInterrupt: 

In [ ]:
invoke_skill_chat_response(token=token)